## <font color=#FF814C>YouTube Trending Videos</font>
**<font color=#B23400>Analyzing Key Features of YouTube Trending Videos from 3 Countries</font>**


The analysis is going to be performed as follows:

- PySpark environment setup
- Loading Dataset and Overview
- Data Profiling and Transformation
- Analyze:
  1. What is the interaction level of the most trending videos in each country?
  2. Do trending Videos have higher like to dislike ratio?
  3. Is there a difference which video types trend per country?
  4. Is there a difference which video types trend per per day of the week?

In [217]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate() 
spark = SparkSession(sc) 

## <font color=#FF814C>I. Data Loading and Overview </font>

### <font color=#FF814C>Data Loading </font>

In this section, the dataset containing youtube trending video data will be read directly from csv and saved in dataframe variable youtube_trending.

The CSV file has the default comma delimiter and the first line of the file is a header. 
To avoid having all data loaded as string, We will load the data and let it infere the datatype and if the correct datatype isn't infered we can manually change it.

In [218]:
from IPython.display import display, Markdown
youtube_trending = spark.read.option("header", "true")\
                        .option("inferSchema", "true")\
                        .csv("US_CA_GBvideos.csv")
youtube_trending.cache()
display(Markdown("We have also cached the dataset so it will remain in memory as we will be calling for the remainder of the analysis.")) 
youtube_trending.printSchema()
youtube_trending.select("trending_date","publish_date").show(2)


We have also cached the dataset so it will remain in memory as we will be calling for the remainder of the analysis.

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- trending_month: integer (nullable = true)
 |-- trending_year: integer (nullable = true)
 |-- t_dow: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- publish_month: integer (nullable = true)
 |-- publish_year: integer (nullable = true)
 |-- p_dow: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- country: string (nullable = true)
 |-- no_tags_used: integer (nullable = true)

+-------------+------------+
|trending_date|publish_date|
+-------------+---------

### <font color=#FF814C>Data Overview </font>
The dataset is loaded correctly and all the variable have been correctly inferred correctly for the rest of variables besides the ones cast above. Thus we can check how many rows and attributes are present, have an overview of the what kind of information is stored in the dataframe by looking at sample rows, and properly determine the structure of the information by identifying what the entities, metrics and dimensions are in our dataset. 

In [213]:
#Identify the dimention of the dataframe (row and column number)
print("The youtube_trending dataset contains", len(youtube_trending.columns), "attributes/columns and", youtube_trending.count(), "records of rows.")

#Taking a sample number of rows to see the information stored. 
sample_youtube_trending=youtube_trending.sample(False, 0.1) #taking random sample of 10% rows from the total dataset
sample_youtube_trending.take(2)

The youtube_trending dataset contains 21 attributes/columns and 120746 records of rows.


[Row(video_id='gHZ1Qz0KiKM', trending_date='11/14/2017', trending_month=11, trending_year=2017, t_dow=2, title='2 Weeks with iPhone X', channel_title='iJustine', category='Science & Technology', publish_date='11/13/2017', publish_month=11, publish_year=2017, p_dow=1, views=119180, likes=9763, dislikes=511, comment_count=1434, comments_disabled=False, ratings_disabled=False, video_error_or_removed=False, country='United States', no_tags_used=7),
 Row(video_id='GgVmn66oK_A', trending_date='11/14/2017', trending_month=11, trending_year=2017, t_dow=2, title='Which Countries Are About To Collapse?', channel_title='NowThis World', category='News & Politics', publish_date='11/12/2017', publish_month=11, publish_year=2017, p_dow=7, views=544770, likes=7848, dislikes=1171, comment_count=3981, comments_disabled=False, ratings_disabled=False, video_error_or_removed=False, country='United States', no_tags_used=25)]

From the information cotained in the rows we can identify which are the entities described, the metrics used to measure these entities and which attibutes add to the dimension of the entity. 
- In this dataset the entity is the trending youtube video which is identified by video_id.
- There are several metrics for this entity such as: **published time, trending date, total number of views, like, comments, and tags used.**
- The attributes that add dimention to the entity are: **country, video category, channel name.**

And as we process towards analyzing the features, it is helpful to identify which columns will be able to give us the information we need and identify if we need any transformation needed. 
The attributes that we have can be categorized as below based on the information they contain
- Video Interactivity Information: **views, likes, dislikes, comment_count**
- Video Discription: **title, category_id, country**
- Timing Information: **trending_date, publish_time**

## <font color=#FF814C>II. Data Profiling and Transformation</font>

Following the metadata analysis of the attributes, this step will go further into outlining and summerizing the data in these attributes individually and between eachother. 
This will allow us:
- to better understand what valuable data there is to answer the business question 
- identify if there is need to transform and broaden our dimension
- detect attributes that will not be useful to answer our business question (remove from cached dataframe)

In [359]:
import pyspark.sql.functions as F

display(Markdown("Checking for nulls on columns video_id, trending_date, publish_time and category")) 
youtube_trending.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ["video_id","trending_date","publish_date","category"]]).show()

display(Markdown("Total number of videos and countries in our dataframe")) 
youtube_trending.select([F.countDistinct(dis).alias(dis) for dis in ["video_id","country","trending_year"]]).show()

display(Markdown("Checking the most frequently and least frequently appearing videos per country")) 
videofrequencyDF = youtube_trending.groupBy("title").agg(F.count(F.lit(1)).alias("Total Occurance"))

video_leastFreq = videofrequencyDF.orderBy(F.col("Total Occurance").asc()).first()
print(video_leastFreq)
video_mostFreq = videofrequencyDF.orderBy(F.col("Total Occurance").desc()).first()
print(video_mostFreq)

display(Markdown("Check the number of days covered in this time frame.")) 
youtube_trending.select([F.countDistinct(c).alias(c) for c in ["publish_date", "trending_date"]]).show()

display(Markdown("Check the types of videos in the dataframe.")) 
youtube_trending.select([F.countDistinct(c).alias(c) for c in ["category"]]).show()


Checking for nulls on columns video_id, trending_date, publish_time and category

+--------+-------------+------------+--------+
|video_id|trending_date|publish_date|category|
+--------+-------------+------------+--------+
|       0|            0|           0|       0|
+--------+-------------+------------+--------+



Total number of videos and countries in our dataframe

+--------+-------+-------------+
|video_id|country|trending_year|
+--------+-------+-------------+
|   29926|      3|            2|
+--------+-------+-------------+



Checking the most frequently and least frequently appearing videos per country

Row(title='FACE SURGERY WITH MAKEUP', Total Occurance=1)
Row(title='Sam Smith - Pray (Official Video) ft. Logic', Total Occurance=70)


Check the number of days covered in this time frame.

+------------+-------------+
|publish_date|trending_date|
+------------+-------------+
|         425|          205|
+------------+-------------+



Check the types of videos in the dataframe.

+--------+
|category|
+--------+
|      17|
+--------+



In [309]:
display(Markdown("what period does our dataframe cover for each country?")) 

period_intial=youtube_trending.select("trending_year","trending_month")\
                          .groupby("trending_year")\
                          .min("trending_month")
    
period_final=youtube_trending.select("trending_year","trending_month")\
                          .groupby("trending_year")\
                          .max("trending_month")

period_intial.orderBy('trending_year',ascending=True).show()
period_final.orderBy('trending_year',ascending=True).show()

what period does our dataframe cover for each country?

+-------------+-------------------+
|trending_year|min(trending_month)|
+-------------+-------------------+
|         2017|                 11|
|         2018|                  1|
+-------------+-------------------+

+-------------+-------------------+
|trending_year|max(trending_month)|
+-------------+-------------------+
|         2017|                 12|
|         2018|                  6|
+-------------+-------------------+



We can see from above that out dataset has data from November of 2017  to June of 2018.

##### Add a column that gives the name day of the week for both trending and published date.

In [232]:
youtube_trending = youtube_trending.withColumn("t_name_month", F.when(F.col("trending_month") == 1, "Jan")\
                                                           .when(F.col("trending_month") == 2, "Feb")\
                                                           .when(F.col("trending_month") == 3, "Mar")\
                                                           .when(F.col("trending_month") == 4, "Apr")\
                                                           .when(F.col("trending_month") == 5, "May")\
                                                           .when(F.col("trending_month") == 6, "Jun")\
                                                           .when(F.col("trending_month") == 7, "Jul")\
                                                           .when(F.col("trending_month") == 8, "Aug")\
                                                           .when(F.col("trending_month") == 9, "Sept")\
                                                           .when(F.col("trending_month") == 10, "Oct")\
                                                           .when(F.col("trending_month") == 11, "Nov")\
                                                           .when(F.col("trending_month") == 12, "Dec")
                                               
youtube_trending = youtube_trending.withColumn("p_name_month", F.when(F.col("publish_month") == 1, "Jan")\
                                                           .when(F.col("publish_month") == 2, "Feb")\
                                                           .when(F.col("publish_month") == 3, "Mar")\
                                                           .when(F.col("publish_month") == 4, "Apr")\
                                                           .when(F.col("publish_month") == 5, "May")\
                                                           .when(F.col("publish_month") == 6, "Jun")\
                                                           .when(F.col("publish_month") == 7, "Jul")\
                                                           .when(F.col("publish_month") == 8, "Aug")\
                                                           .when(F.col("publish_month") == 9, "Sept")\
                                                           .when(F.col("publish_month") == 10, "Oct")\
                                                           .when(F.col("publish_month") == 11, "Nov")\
                                                           .when(F.col("trending_month") == 12, "Dec")

youtube_trending.cache()
youtube_trending.groupby("trending_month","t_name_month","publish_month","p_name_month").show(2)

+--------------+------------+-------------+------------+
|trending_month|t_name_month|publish_month|p_name_month|
+--------------+------------+-------------+------------+
|            11|         Nov|           11|         Nov|
|            11|         Nov|           11|         Nov|
+--------------+------------+-------------+------------+
only showing top 2 rows



##### Add a column that gives the name month for both trending and published date.

In [225]:
youtube_trending = youtube_trending.withColumn("t_name_dow", F.when(F.col("t_dow") == 1, "Monday")\
                                                           .when(F.col("t_dow") == 2, "Tuesday")\
                                                           .when(F.col("t_dow") == 3, "Wednesday")\
                                                           .when(F.col("t_dow") == 4, "Thursday")\
                                                           .when(F.col("t_dow") == 5, "Friday")\
                                                           .when(F.col("t_dow") == 6, "Saturday")\
                                                           .otherwise("Sunday"))
youtube_trending = youtube_trending.withColumn("p_name_dow", F.when(F.col("p_dow") == 1, "Monday")\
                                                           .when(F.col("p_dow") == 2, "Tuesday")\
                                                           .when(F.col("p_dow") == 3, "Wednesday")\
                                                           .when(F.col("p_dow") == 4, "Thursday")\
                                                           .when(F.col("p_dow") == 5, "Friday")\
                                                           .when(F.col("p_dow") == 6, "Saturday")\
                                                           .otherwise("Sunday"))

youtube_trending.cache()
youtube_trending.select("t_dow","t_name_dow","p_dow","p_name_dow").show(2)

+-----+----------+-----+----------+
|t_dow|t_name_dow|p_dow|p_name_dow|
+-----+----------+-----+----------+
|    2|   Tuesday|    1|    Monday|
|    2|   Tuesday|    1|    Monday|
+-----+----------+-----+----------+
only showing top 2 rows



#### Add a column that calculates the different in day from when a video was published to when it started trending.

In [ ]:
youtube_trending = youtube_trending.withColumn("t_name_dow", F.when(F.col("t_dow") == 1, "Monday")\
                                                           .when(F.col("t_dow") == 2, "Tuesday")\
                                                           .when(F.col("t_dow") == 3, "Wednesday")\
                                                           .when(F.col("t_dow") == 4, "Thursday")\
                                                           .when(F.col("t_dow") == 5, "Friday")\
                                                           .when(F.col("t_dow") == 6, "Saturday")\
                                                           .otherwise("Sunday"))
youtube_trending.cache()

## <font color=#FF814C>II. Data Analysis</font>

### <font color=#FF4B00>Total Trending Videos per Month for CA, US and GB</font>

In [258]:
display(Markdown("Calculate the count of distinct videos that were ")) 
tvcountrymmyy = youtube_trending.select("t_name_month", "trending_month","video_id","country","trending_year")\
                       .groupBy("country","trending_year","t_name_month","trending_month")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos"))

cols = ['country','trending_year','trending_month']
tvcountrymmyy.orderBy(cols,ascending=True).show() #ordering/sorting the resulting dataframe
print("The tvcountrymmyy dataframe contains", len(tvcountrymmyy.columns), "attributes/columns and", tvcountrymmyy.count(), "records of rows.")
tvcountrymmyy.cache()

display(Markdown("Which country had the largest number of videos trending?")) 
youtube_trending.select("video_id","country")\
                       .groupBy("country")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos")).show()


Calculate the count of distinct videos that were 

+-------------+-------------+------------+--------------+----------------------+
|      country|trending_year|t_name_month|trending_month|DistinctTrendingVideos|
+-------------+-------------+------------+--------------+----------------------+
|       Canada|         2017|         Nov|            11|                  1990|
|       Canada|         2017|         Dec|            12|                  3545|
|       Canada|         2018|         Jan|             1|                  3445|
|       Canada|         2018|         Feb|             2|                  3327|
|       Canada|         2018|         Mar|             3|                  3790|
|       Canada|         2018|         Apr|             4|                  3044|
|       Canada|         2018|         May|             5|                  3842|
|       Canada|         2018|         Jun|             6|                  1776|
|Great Britain|         2017|         Nov|            11|                   561|
|Great Britain|         2017

Which country had the largest number of videos trending?

+-------------+----------------------+
|      country|DistinctTrendingVideos|
+-------------+----------------------+
|Great Britain|                  3237|
|United States|                  6282|
|       Canada|                 24104|
+-------------+----------------------+



### <font color=#FF4B00>Trending Videos per Category Type</font>

In [322]:
display(Markdown("Calculate how many video are trending for each catagory in total for the whole period.")) 
video_category = youtube_trending.select("video_id","country","category","trending_year")\
                       .groupBy("category")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos"))

cols = ['country','trending_year',"DistinctTrendingVideos"]
video_category.orderBy("DistinctTrendingVideos",ascending=False).show() #sort from highest to lowest count

display(Markdown("Check if the top trending video type in each country per year alligns with the general results.")) 
display(Markdown("CANADA")) 
video_category_CA = youtube_trending.select("video_id","country","category","trending_year")\
                       .where((F.col("country")=='Canada'))\
                       .groupBy("category")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos"))
video_category_CA.orderBy("DistinctTrendingVideos",ascending=False).show(3)


display(Markdown("UNITED STATES")) 
video_category_US = youtube_trending.select("video_id","country","category","trending_year")\
                       .where((F.col("country")=='United States'))\
                       .groupBy("category")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos"))
video_category_US.orderBy("DistinctTrendingVideos",ascending=False).show(3)

display(Markdown("GREAT BRITAIN")) 
video_category_GB = youtube_trending.select("video_id","country","category","trending_year")\
                       .where((F.col("country")=='Great Britain'))\
                       .groupBy("category")\
                       .agg(F.countDistinct("video_id").alias("DistinctTrendingVideos"))
video_category_GB.orderBy("DistinctTrendingVideos",ascending=False).show(3)


Calculate how many video are trending for each catagory in total for the whole period.

+--------------------+----------------------+
|            category|DistinctTrendingVideos|
+--------------------+----------------------+
|       Entertainment|                  9612|
|     News & Politics|                  3377|
|      People & Blogs|                  3036|
|               Music|                  2458|
|              Sports|                  2400|
|              Comedy|                  2280|
|       Howto & Style|                  1759|
|    Film & Animation|                  1415|
|              Gaming|                   953|
|Science & Technology|                   887|
|           Education|                   753|
|      Pets & Animals|                   323|
|    Autos & Vehicles|                   307|
|     Travel & Events|                   243|
|               Shows|                   113|
|Nonprofits & Acti...|                    67|
|              Movies|                     1|
+--------------------+----------------------+



Check if the top trending video type in each country per year alligns with the general results.

CANADA

+---------------+----------------------+
|       category|DistinctTrendingVideos|
+---------------+----------------------+
|  Entertainment|                  8140|
|News & Politics|                  2910|
| People & Blogs|                  2531|
+---------------+----------------------+
only showing top 3 rows



UNITED STATES

+-------------+----------------------+
|     category|DistinctTrendingVideos|
+-------------+----------------------+
|Entertainment|                  1610|
|        Music|                   796|
|Howto & Style|                   591|
+-------------+----------------------+
only showing top 3 rows



GREAT BRITAIN

+--------------+----------------------+
|      category|DistinctTrendingVideos|
+--------------+----------------------+
|         Music|                   873|
| Entertainment|                   850|
|People & Blogs|                   283|
+--------------+----------------------+
only showing top 3 rows



### <font color=#FF4B00>Do certain video catagories trend of specific day of the week? </font>

In [381]:
display(Markdown("**Day of the Week and total videos Trending per category**"))

youtube_trending.agg(F.countDistinct("video_id")).show()


category_dow=youtube_trending.groupBy("category")\
              .pivot("p_name_dow")\
              .agg(F.countDistinct("video_id").alias("Number of Videos Trending"))

category_dow.cache()
category_dow.orderBy("category").show()

#category_dow2= youtube_trending.groupBy("category")\
                #.pivot("p_name_dow")\
                #.agg(((F.countDistinct("video_id"))/29926*100).alias("totalvideo%"))




**Day of the Week and total videos Trending per category**

+------------------------+
|count(DISTINCT video_id)|
+------------------------+
|                   29926|
+------------------------+

+--------------------+------+------+--------+------+--------+-------+---------+
|            category|Friday|Monday|Saturday|Sunday|Thursday|Tuesday|Wednesday|
+--------------------+------+------+--------+------+--------+-------+---------+
|    Autos & Vehicles|    33|    46|      18|    35|      83|     60|       36|
|              Comedy|   425|   289|     248|   262|     351|    358|      353|
|           Education|    82|   148|      95|   106|     128|    123|       76|
|       Entertainment|  1570|  1485|    1218|  1042|    1393|   1459|     1451|
|    Film & Animation|   193|   205|     200|   179|     220|    229|      195|
|              Gaming|   138|   127|     154|   165|     138|    107|      128|
|       Howto & Style|   262|   205|     191|   289|     281|    268|      269|
|              Movies|  null|  null|    null|  null|       1|   

### <font color=#FF4B00>Video Interactivity of Trending Videos Per Category</font>

In [415]:
youtube_trending.groupBy("category")\
              .agg(F.max("views").alias("AverageViews"),\
                   F.max("dislikes").alias("AverageDislikes"),\
                   F.max("likes").alias("AverageLikes"))\
                    .orderBy("category").show()

+--------------------+------------+---------------+------------+
|            category|AverageViews|AverageDislikes|AverageLikes|
+--------------------+------------+---------------+------------+
|    Autos & Vehicles|    25244097|          12647|      114841|
|              Comedy|    43460605|          43698|     1253154|
|           Education|    12100921|          13561|      383049|
|       Entertainment|   169884583|        1944971|     3312868|
|    Film & Animation|    54863912|          60025|      922355|
|              Gaming|    18158133|         212976|     1031695|
|       Howto & Style|    54155921|          66632|      605954|
|              Movies|     5661965|           3784|       67232|
|               Music|   424538912|         421473|     5613827|
|     News & Politics|    18994966|         132439|      610526|
|Nonprofits & Acti...|    26703269|         545595|     2061275|
|      People & Blogs|    62338362|         504340|     2032463|
|      Pets & Animals|   